# Lab 05: Bedrock Guardrails

## Overview

In this notebook, we add **Bedrock Guardrails** to filter out off-topic and harmful queries before they reach the LLM. This saves tokens on irrelevant requests and improves safety.

**What you'll learn:**
- How to create a Bedrock Guardrail
- How to configure topic and content filters
- How to integrate guardrails with your agent
- Token savings from blocked queries

**Guardrail Filters:**
- Topic filters: Block competitor questions, investment advice, medical advice
- Content filters: Block violence, hate speech

## Prerequisites

- Completed Labs 01-04

## Workshop Journey

```
01 Baseline → 02 Quick Wins → 03 Caching → 04 Routing → [05 Guardrails] → 06 Gateway → 07 Evaluations
                                                             ↑
                                                        You are here
```

## Step 1: Setup

In [ ]:
import os
import json
import uuid
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
bedrock_client = boto3.client("bedrock", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_HOST', 'https://cloud.langfuse.com')}")

## Step 2: Create a Bedrock Guardrail

We'll create a guardrail that blocks:
- Questions about competitor products
- Investment/financial advice requests
- Medical advice requests
- Violent or hateful content

In [ ]:
# Create the guardrail
guardrail_name = "customer-support-guardrail"

try:
    guardrail_response = bedrock_client.create_guardrail(
        name=guardrail_name,
        description="Block off-topic and harmful queries for customer support agent",
        blockedInputMessaging="I'm sorry, but this question is outside my scope as a TechMart customer support assistant. I can help you with product information, returns, and technical support.",
        blockedOutputsMessaging="I apologize, but I cannot provide that type of information. Please let me know if you have questions about TechMart products or services.",
        topicPolicyConfig={
            "topicsConfig": [
                {
                    "name": "competitor-products",
                    "definition": "Questions comparing TechMart to competitors like Apple, Samsung, Dell, HP, or asking which brand is better",
                    "examples": [
                        "Is your laptop better than a MacBook?",
                        "Should I buy from you or Best Buy?",
                        "How do you compare to Apple?"
                    ],
                    "type": "DENY"
                },
                {
                    "name": "investment-advice",
                    "definition": "Questions about financial investments, stock market, cryptocurrency, or financial planning",
                    "examples": [
                        "Should I invest in tech stocks?",
                        "Is now a good time to buy crypto?",
                        "What stocks do you recommend?"
                    ],
                    "type": "DENY"
                },
                {
                    "name": "medical-advice",
                    "definition": "Questions about medical diagnoses, treatments, or health conditions",
                    "examples": [
                        "Can screen time cause headaches?",
                        "Is blue light bad for my eyes?",
                        "What medicine should I take?"
                    ],
                    "type": "DENY"
                }
            ]
        },
        contentPolicyConfig={
            "filtersConfig": [
                {"type": "VIOLENCE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
                {"type": "HATE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
                {"type": "INSULTS", "inputStrength": "MEDIUM", "outputStrength": "MEDIUM"},
            ]
        },
    )
    
    guardrail_id = guardrail_response["guardrailId"]
    print(f"Guardrail created: {guardrail_id}")
    
except bedrock_client.exceptions.ConflictException:
    # Guardrail already exists, get its ID
    response = bedrock_client.list_guardrails()
    for g in response["guardrails"]:
        if g["name"] == guardrail_name:
            guardrail_id = g["id"]
            print(f"Using existing guardrail: {guardrail_id}")
            break

In [ ]:
# Save guardrail ID for later
print(f"Guardrail ID: {guardrail_id}")
print("\nAdd this to your .env file:")
print(f"GUARDRAIL_ID={guardrail_id}")

## Step 3: Review the Guardrails Agent

In [ ]:
# Review the v5 agent code
agent_file = Path("agents/v5_guardrails.py")
print(agent_file.read_text())

## Step 4: Deploy the Guardrails Agent

In [ ]:
agent_name = "customer_support_v5_guardrails"
agent_file = str(Path("agents/v5_guardrails.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

In [ ]:
# Modify Dockerfile for Langfuse
import re

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    if "opentelemetry-instrument" in content:
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]',
            r'CMD ["python", "-m", "\1"]',
            content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")

In [ ]:
env_vars = {
    "LANGFUSE_HOST": os.environ.get("LANGFUSE_HOST"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "GUARDRAIL_ID": guardrail_id,
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

## Step 5: Test Guardrails

In [ ]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [ ]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    get_latest_trace_metrics,
    print_metrics,
    clear_metrics,
    collect_metric,
    print_metrics_table,
    get_collected_metrics
)

# Clear any previously collected metrics
clear_metrics()
print("Metrics helper ready")

In [ ]:
# Standard test prompts - each demonstrates a specific tool usage pattern
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),

    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),

    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),

    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),

    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
print("=" * 60)
print("VALID QUERIES (Should Pass Through Guardrails)")
print("=" * 60)

for test_name, prompt in TEST_PROMPTS:
    print("\n" + "=" * 60)
    print(f"Test: {test_name}")
    print("=" * 60)

    response = invoke_agent(prompt)
    print(f"Response: {response}")

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v5-guardrails",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

In [ ]:
# Blocked queries - demonstrate guardrail filtering
# Note: These queries should be blocked BEFORE reaching the LLM, saving 100% of tokens
print("=" * 60)
print("BLOCKED QUERIES (Should Be Filtered by Guardrails)")
print("=" * 60)
print("Note: Blocked queries save tokens by filtering BEFORE LLM processing")

blocked_queries = [
    ("Competitor Comparison", "Is your laptop better than a MacBook?"),
    ("Investment Advice", "Should I invest in tech stocks?"),
    ("Brand Comparison", "What's better, your phone or the iPhone?"),
    ("Crypto Advice", "Can you recommend any cryptocurrency?"),
]

for test_name, query in blocked_queries:
    print(f"\n--- {test_name} ---")
    print(f"Query: {query}")
    result = invoke_agent(query)
    print(f"Response: {str(result)[:200]}")

In [ ]:
# Print summary table and comparison vs baseline
print_metrics_table()

# Baseline metrics from notebook 01
BASELINE_AVG_INPUT_TOKENS = 4251
BASELINE_AVG_LATENCY = 8.0

# Calculate improvements
collected = get_collected_metrics()
if collected:
    valid_metrics = [m for m in collected if "error" not in m]
    if valid_metrics:
        avg_input = sum(m.get('input_tokens', 0) for m in valid_metrics) / len(valid_metrics)
        avg_latency = sum(m.get('latency_seconds', 0) or 0 for m in valid_metrics) / len(valid_metrics)

        token_reduction = ((BASELINE_AVG_INPUT_TOKENS - avg_input) / BASELINE_AVG_INPUT_TOKENS) * 100
        latency_change = ((BASELINE_AVG_LATENCY - avg_latency) / BASELINE_AVG_LATENCY) * 100

        print("\n" + "=" * 60)
        print("           COMPARISON VS BASELINE (v1)")
        print("=" * 60)
        print(f"  Avg Input Tokens:  {avg_input:,.0f} (Baseline: {BASELINE_AVG_INPUT_TOKENS:,})")
        print(f"  Token Reduction:   {token_reduction:+.1f}%")
        print(f"  Avg Latency:       {avg_latency:.2f}s (Baseline: {BASELINE_AVG_LATENCY:.2f}s)")
        print(f"  Latency Change:    {latency_change:+.1f}%")
        print("=" * 60)
        print("\n📝 Note: Guardrails also save tokens on blocked queries (not shown above)")
        print("   Blocked queries consume 0 LLM tokens - instant cost savings!")

## Step 6: Analyze Guardrail Impact in Langfuse

In [ ]:
langfuse_host = os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")
print(f"View your traces at: {langfuse_host}")
print("\nFilter by tags: 'guardrails'")
print("\n" + "=" * 60)
print("Expected Behavior:")
print("=" * 60)
print("")
print("| Query Type | LLM Tokens Used | Response |")
print("|------------|-----------------|----------|")
print("| Valid      | Yes (normal)    | Full answer |")
print("| Blocked    | No (saved!)     | Guardrail message |")
print("")
print("Blocked queries save 100% of LLM tokens!")

## Summary

In this notebook, we added Bedrock Guardrails:

1. **Topic filters**: Block competitor comparisons, investment/medical advice
2. **Content filters**: Block violent and hateful content
3. **Token savings**: Blocked queries don't consume LLM tokens

**Benefits:**
- Cost savings on off-topic queries
- Improved safety and compliance
- Consistent messaging for out-of-scope requests

**Next Steps:** In the final optimization notebook, we'll add AgentCore Gateway for semantic tool search.

**Next notebook:** [06-agentcore-gateway.ipynb](./06-agentcore-gateway.ipynb)

## Cleanup (Optional)

In [ ]:
# Uncomment to delete the guardrail
# bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
# print(f"Deleted guardrail: {guardrail_id}")